In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

### The interesting datasets seems to be:
        * condition_occurrence : where all the diagnosis can symptoms are stored
        * observation: test and measurements done
        * measurement_occurrence_1
        * precedure_occurene_1

In [2]:
csv_directory = "/home/ubuntu/zehai/data/synpuf/"
json_directory = "/home/ubuntu/zehai/data/AchillesExport/"

In [3]:
## Load the condition_occation file from the directory
condition_occur = pd.read_csv(csv_directory +"condition_occurrence.csv",dtype={'condition_concept_id':'int64',
 'condition_end_date': 'O',
 'condition_occurrence_id':'int64',
 'condition_source_concept_id':'int64',
 'condition_source_value':'O',
 'condition_start_date':'O',
 'condition_type_concept_id':'int64',
 'person_id':'int64',
 'provider_id':'float64',
 'stop_reason': 'float64',
 'visit_occurrence_id': 'int64'}, nrows = 43367979 )

In [4]:
#read the condition map from Json file
condition_map = pd.read_json(json_directory+"condition_treemap.json")

In [5]:
# read the observation file
observation = pd.read_csv(csv_directory+"observation.csv", dtype = {'observation_concept_id': 'int64',
 'observation_date': 'O',
 'observation_id': 'int64',
 'observation_source_concept_id': 'int64',
 'observation_source_value': 'O',
 'observation_time': 'float64',
 'observation_type_concept_id': 'int64',
 'person_id': 'int64',
 'provider_id': 'float64',
 'qualifier_concept_id': 'float64',
 'qualifier_source_value': 'float64',
 'unit_concept_id': 'float64',
 'unit_source_value': 'float64',
 'value_as_concept_id': 'int64',
 'value_as_number': 'float64',
 'value_as_string': 'float64',
 'visit_occurrence_id': 'int64'})

In [6]:
# Load brain antonmy keys from txt fiel
with open('brain_antonomy_keys.txt') as f:
     read_data = f.read()
f.closed
antonomy_regions = [x.strip() for x in read_data.split(',')]

In [7]:
# read the observation map
ob_map = pd.read_json(json_directory+"observation_treemap.json")

In [8]:
# conditionera map from Json dir
conditionera_map = pd.read_json(json_directory+"conditionera_treemap.json")

In [9]:
# read the Head MRI related patient from the oberservation dataset
#      CONCEPT_ID                                       CONCEPT_PATH  \   
# 121     2106371  Radiography of head||Regional contrast radiolo...   
# 218     2106382  Radiography of head||Regional contrast radiolo...     

#      NUM_PERSONS  PERCENT_PERSONS  RECORDS_PER_PERSON   
# 121         1137         0.009772            1.117854  
# 218          519         0.004461            1.015414   

brain_identify_patient = observation[(observation['observation_concept_id']==2106371)
                                     |(observation['observation_concept_id']==2106382)]
patient_id = brain_identify_patient['person_id'].drop_duplicates()

In [10]:
# select all the patients with head mri scan history
mri_condition_occur = condition_occur[condition_occur['person_id'].isin(list(patient_id))]

In [11]:
# de_duplicate condition_concept, select all the symptoms related to the MRI patients
mri_conditions = mri_condition_occur.loc[:,['person_id','condition_concept_id']]
head_condition = mri_conditions.condition_concept_id.drop_duplicates()
mri_related_symptom = condition_map['CONCEPT_PATH'][condition_map.CONCEPT_ID.isin(list(head_condition))]

In [12]:
# load brain antonmy keys from txt fiel
with open('brain_antonomy_keys.txt') as f:
     read_data = f.read()
f.closed
antonomy_regions = [x.strip() for x in read_data.split(',')]

In [14]:
# converted everything to lower case for matching
antonomy_regions = [x.lower() for x in antonomy_regions]
conditionera_map['CONCEPT_PATH'] = conditionera_map['CONCEPT_PATH'].str.lower()

In [15]:
# match the conditionera SNOMED discriptions with the brain section regions
brain_related_map = conditionera_map.loc[:,['CONCEPT_ID','CONCEPT_PATH']]\
[sum([conditionera_map['CONCEPT_PATH'].str.contains(x) for x in antonomy_regions])>0]

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [16]:
# select all mri patient with final brain issue diagnosis
brain_related_list = list(brain_related_map['CONCEPT_ID'])
brain_problem_diagnosis = mri_conditions.loc[:,['person_id','condition_concept_id']]\
[mri_conditions['condition_concept_id'].isin(brain_related_list)]
brain_problem_patients = brain_problem_diagnosis.drop_duplicates()
brain_problem_diagnosis['diagnosis'] = 0
brain_related_list = list(brain_related_map['CONCEPT_ID'])
brain_diagnosis = list(brain_related_map['CONCEPT_PATH'].str.split('|', expand=True)[8])
for i in range(len(brain_related_list)):
    brain_problem_diagnosis['diagnosis'][brain_problem_diagnosis['condition_concept_id']== brain_related_list[i]] = brain_diagnosis[i]

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
# selected all symptoms related to brain issue
mri_symptoms = mri_conditions[mri_conditions.condition_concept_id.isin(brain_related_list)==False].drop_duplicates()
symptoms_of_braindamage = mri_symptoms.condition_concept_id.drop_duplicates()
symptoms_of_braindamage.head()

12823    42872402
12824      437827
12825      321588
12826      434005
12827      140673
Name: condition_concept_id, dtype: int64

In [18]:
# selected specific symptom with brain issue population
symptomlist = condition_map.loc[:,['CONCEPT_ID','CONCEPT_PATH']][condition_map['CONCEPT_ID'].isin(list(symptoms_of_braindamage))]
symptomlist = symptomlist.reset_index()
symptomlist.head()

,index,CONCEPT_ID,CONCEPT_PATH
0,0,201826,NA||NA||NA||NA||Type 2 diabetes mellitus
1,1,437827,NA||NA||NA||NA||Pure hypercholesterolemia
2,2,140673,NA||NA||NA||NA||Hypothyroidism
3,3,313217,NA||NA||NA||NA||Atrial fibrillation
4,4,439926,NA||NA||NA||NA||Malaise and fatigue


In [48]:
# read in all the neurosynth dictionary
with open('/home/ubuntu/zehai/data/NS_TERMS.txt') as f:
     read_data = f.read()
f.closed
neuro_keylist = [x.strip() for x in read_data.split('\n')]

In [49]:
brain_problem_diagnosis_deduplic.head()

,person_id,condition_concept_id,diagnosis
246025,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...
421191,3332,377842,frontal lobe syndrome
561170,4359,4181481,malignant neoplasm of parietal lobe
736777,5780,434821,systemic inflammatory response syndrome
839888,6591,377842,frontal lobe syndrome


In [50]:
patient_symptoms.head()

0    [42872402, 437827, 321588, 434005, 140673, 319...
1    [4324765, 318800, 73754, 74464, 192357, 28974,...
2    [78804, 80767, 77030, 75617, 72578, 200970, 43...
3    [259043, 318800, 437827, 80180, 313217, 416146...
4    [256449, 201826, 319835, 436665, 313878, 26012...
Name: condition_concept_id, dtype: object

In [52]:
patient_symptoms = mri_symptoms.groupby('person_id')['condition_concept_id'].apply(list).reset_index()
brain_problem_diagnosis_deduplic = brain_problem_diagnosis.drop_duplicates()
final_parse = pd.merge(brain_problem_diagnosis_deduplic, patient_symptoms, on='person_id', how='inner')
final_parse.head()
final_parse.rename(columns={'condition_concept_id_x': 'diagnosis_id', 'condition_concept_id_y': 'symptom_id_list'}, inplace=True)
final_parse.head()

,person_id,diagnosis_id,diagnosis,symptom_id_list
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,"[440977, 42872402, 443597, 137829, 201826, 376..."
1,3332,377842,frontal lobe syndrome,"[201072, 433736, 201826, 377091, 319835, 37432..."
2,4359,4181481,malignant neoplasm of parietal lobe,"[319049, 440385, 374022, 193688, 81902, 435141..."
3,5780,434821,systemic inflammatory response syndrome,"[444406, 438624, 40480602, 441829, 42872402, 4..."
4,6591,377842,frontal lobe syndrome,"[444406, 132797, 42872402, 4112183, 318800, 38..."


In [89]:
def keys_append(concept_path,neuro_keylist):
    # append neurosynth keys to symptom map
    concept_path = concept_path.lower()
    keylist = []
    for x in neuro_keylist:
        # String length greater than 3 is totally arbitrary
        # But it makes the resulting lists look way better, so
        # For the sake of expediency, we're going with it
        # TODO: get a better heuristic here
        if concept_path.find(' ' + x) != -1 and len(x) > 3:
            keylist.append(x)
    #keylist = ', '.join(keylist)
    return keylist

symp_list = []
for i in list(symptomlist.CONCEPT_PATH):
    #list of keys concatenated
    symp_list.append(keys_append(i, neuro_keylist))

In [90]:
# Add symptom key_list to patient table
sl = pd.Series( (v for v in symp_list) )
# symptomlist = symptomlist.drop(columns = 'index')
symptomlist['keys_list'] = sl
symptomlist.tail()

,index,CONCEPT_ID,CONCEPT_PATH,keys_list
6632,7128,193136,NA||NA||NA||NA||Acute gonococcal cervicitis,[]
6633,7130,4237906,NA||NA||NA||NA||Poisoning by caffeine,[]
6634,7134,194288,NA||NA||NA||NA||Chlamydial infection of anus a...,[]
6635,7136,374020,NA||NA||NA||NA||Infective encephalitis,[]
6636,7137,434509,NA||NA||NA||NA||Injury of duodenum without ope...,[open]


In [91]:
len(keylist)
list(symptomlist['keys_list'][symptomlist['CONCEPT_ID']==434509])[0]


NameError: name 'keylist' is not defined

In [92]:
def key_match(keylist,symptomlist):
    # match all the keylist with the symptom to neurosynth dict
    # translate symptoms to key
    keys_per_symp = []
    for i in keylist:
        thinging = list(symptomlist['keys_list'][symptomlist['CONCEPT_ID']==i])
        if thinging==[]:
            keys_per_symp.append(thinging)
        else:
            keys_per_symp.append(thinging[0])
    return keys_per_symp

## test on the first patient
# keylist = final_parse.symptom_id_list[0]
# key_match(keylist,symptomlist)

# type(symptomlist['keys_list'][symptomlist['CONCEPT_ID']==440977])

keys_per_person =[]
for x in list(final_parse.symptom_id_list):
        keys_per_person.append(key_match(x,symptomlist))

In [93]:
ssl = pd.Series( (v for v in keys_per_person) )
final_parse['neurosynth_keys'] = ssl
final_parse.head()

,person_id,diagnosis_id,diagnosis,symptom_id_list,neurosynth_keys
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,"[440977, 42872402, 443597, 137829, 201826, 376...","[[], [native], [disease, stage], [], [], [tiss..."
1,3332,377842,frontal lobe syndrome,"[201072, 433736, 201826, 377091, 319835, 37432...","[[], [], [], [], [failure, heart], [dementia, ..."
2,4359,4181481,malignant neoplasm of parietal lobe,"[319049, 440385, 374022, 193688, 81902, 435141...","[[failure], [], [], [], [disease, tract], [pro..."
3,5780,434821,systemic inflammatory response syndrome,"[444406, 438624, 40480602, 441829, 42872402, 4...","[[], [], [chronic, failure, heart], [], [nativ..."
4,6591,377842,frontal lobe syndrome,"[444406, 132797, 42872402, 4112183, 318800, 38...","[[], [], [native], [disorder], [disease], [], ..."


In [94]:
symp_map = symptomlist.drop(columns='index')
symp_map.head()


,CONCEPT_ID,CONCEPT_PATH,keys_list
0,201826,NA||NA||NA||NA||Type 2 diabetes mellitus,[]
1,437827,NA||NA||NA||NA||Pure hypercholesterolemia,[]
2,140673,NA||NA||NA||NA||Hypothyroidism,[]
3,313217,NA||NA||NA||NA||Atrial fibrillation,[]
4,439926,NA||NA||NA||NA||Malaise and fatigue,[]


In [95]:
# expand all symptoms
symptom_list = final_parse.loc[:,['person_id','symptom_id_list']]
symptom_list.head()

neuro_list = final_parse.loc[:,['person_id','neurosynth_keys']]
neuro_list.head()

s = symptom_list.apply(lambda x: pd.Series(x['symptom_id_list']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'symptom_id'
s = s.apply(int)

n = neuro_list.apply(lambda x: pd.Series(x['neurosynth_keys']),axis=1).stack().reset_index(level=1, drop=True)
n.name = 'neurosynth_key'
n.head()

combine = pd.concat([s, n], axis=1)
symptom_list.head()

second_parse = final_parse.loc[:,['person_id','diagnosis_id','diagnosis']]

second_parse = second_parse.join(combine)
second_parse.head()

,person_id,diagnosis_id,diagnosis,symptom_id,neurosynth_key
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,440977,[]
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,42872402,[native]
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,443597,"[disease, stage]"
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,137829,[]
0,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,201826,[]


In [102]:
# delete all symptoms with no neurosynth key match
# third_parse = second_parse[second_parse['neurosynth_key'].apply(len())]
second_parse = second_parse.reset_index(drop=True)
def isempty(x):
    if x==[]:
        return False
    else:
        return True

third_parse = second_parse[second_parse['neurosynth_key'].apply(isempty)]
third_parse.count()

person_id         44256
diagnosis_id      44256
diagnosis         44256
symptom_id        44256
neurosynth_key    44256
dtype: int64

In [103]:
to_save = third_parse.merge(symp_map.loc[:,['CONCEPT_ID','CONCEPT_PATH']].rename(columns={'CONCEPT_ID':'symptom_id'}), how='left', on='symptom_id')

In [104]:
to_save['symptom_name'] = to_save.CONCEPT_PATH.str.split('|').str[-1]

In [105]:
! mkdir ./clint/docs/example_data

mkdir: cannot create directory ‘./clint/docs/example_data’: File exists


In [107]:
to_save.to_pickle('./clint/docs/example_data/symptom_table_1.pkz')

In [108]:
# Get a list of all symptom feature name
# symp_map['CONCEPT_PATH'][symp_map['CONCEPT_ID']==193136]
symp_name = []
for i in list(third_parse['symptom_id']):
    symp_name.append(list(symp_map['CONCEPT_PATH'][symp_map['CONCEPT_ID']==i]))

In [109]:
symp_name_list = pd.Series( (v for v in symp_name) )
symp_name_list.head()

0    [NA||NA||NA||NA||Coronary arteriosclerosis in ...
1     [NA||NA||NA||NA||Chronic kidney disease stage 3]
2            [NA||NA||NA||NA||Disorder of soft tissue]
3    [NA||NA||NA||NA||Intra-abdominal and pelvic sw...
4    [NA||NA||NA||NA||Malignant neoplasm of skin of...
dtype: object

In [110]:
neurosynth_parse = final_parse.drop(columns = ['diagnosis_id'])
neurosynth_parse.tail()

,person_id,diagnosis,symptom_id_list,neurosynth_keys
590,347655,dementia of frontal lobe type,"[440556, 437390, 434894, 261880, 193493, 43521...","[[], [], [post], [], [disease], [acute, residu..."
591,347655,frontal lobe syndrome,"[440556, 437390, 434894, 261880, 193493, 43521...","[[], [], [post], [], [disease], [acute, residu..."
592,347945,malignant neoplasm of parietal lobe,"[373503, 380378, 317898, 4108356, 443211, 3728...","[[cerebral], [], [essential], [cerebral], [], ..."
593,348369,primary malignant neoplasm of cerebellum,"[40481762, 140673, 319835, 374022, 436222, 437...","[[disease, late], [], [failure, heart], [], [m..."
594,348416,dementia of frontal lobe type,"[313878, 441258, 42872402, 318800, 439697, 433...","[[symptom], [disease], [native], [disease], [d..."


In [111]:
# Clean all the empty symptom keys
import pickle as pk

file_Name = "patient_symptoms_3"
# open the file for writing
fileObject = open(file_Name,'wb') 

# this writes the object a to the
# file named 'testfile'
# pk.dump(neurosynth_parse,fileObject)   
pk.dump(third_parse,fileObject)   

# here we close the fileObject
fileObject.close()

In [112]:
import pickle

def get_label_map():
    levels = ['region', 'hemisphere_region', 'lobe', 'gyrus', 'tissue', 'sub-label']
    label_map = {}
    for level in levels:
        with open ('/home/ubuntu/zehai/clint/data/%s_label_map.pkz'%level, 'rb') as h:
            label_map.update(pickle.load( h))
    return label_map
brain_label_map = get_label_map()

In [113]:
# convert the map list to lower case
map_list = [x for x in list(brain_label_map.keys())]
a = map_list[0]
print (a)
# brain_label_map[a]

map_dim = np.shape(brain_label_map[a])

Brodmann area 23


In [114]:
third_parse.head()

,person_id,diagnosis_id,diagnosis,symptom_id,neurosynth_key
1,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,42872402,[native]
2,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,443597,"[disease, stage]"
5,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,376208,[tissue]
8,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,4168222,[mass]
9,1928,137519,thyrotoxicosis from ectopic thyroid nodule wit...,4162993,"[limb, skin]"


In [125]:
test_set = third_parse[:10]
test_set.count()

person_id         10
diagnosis_id      10
diagnosis         10
symptom_id        10
neurosynth_key    10
dtype: int64

In [124]:
map_list

['Brodmann area 23',
 'Brodmann area 18',
 'Brodmann area 8',
 'Fastigium',
 'Inferior Parietal Lobule',
 'Brodmann area 41',
 'Culmen',
 'Brodmann area 39',
 'Transverse Temporal Gyrus',
 'Brodmann area 27',
 'Brodmann area 25',
 'Lateral Globus Pallidus',
 'Ventral Posterior Medial Nucleus',
 'Left Brainstem',
 'Brodmann area 47',
 'Cerebro-Spinal Fluid',
 'Caudate Body',
 'Tuber of Vermis',
 'Temporal Lobe',
 'Inferior Occipital Gyrus',
 'Brodmann area 29',
 'Brodmann area 40',
 'Inter-Hemispheric',
 'Brodmann area 31',
 'Brodmann area 44',
 'Postcentral Gyrus',
 'Brodmann area 38',
 'Brodmann area 21',
 'Brodmann area 9',
 'Cuneus',
 'Superior Frontal Gyrus',
 'Left Cerebrum',
 'Anterior Nucleus',
 'Medial Geniculum Body',
 'Supramarginal Gyrus',
 'Cerebrum',
 'Pulvinar',
 'Hippocampus',
 'Brodmann area 33',
 'Cerebellar Lingual',
 'Dentate',
 'Cingulate Gyrus',
 'Uvula of Vermis',
 'Precuneus',
 'Putamen',
 'Frontal Lobe',
 'Brodmann area 43',
 'Parahippocampal Gyrus',
 'Insula',


In [132]:
not not check_share_lexi('Brodmann area 23','Brodmannnnn area 25')

True

In [135]:
# Map the brain diagnosis with brain map
# for i in map_list: # loop over the entier map_list

def check_share_lexi(a,b):
    # check if two string lists overlap in a word sense
    a = a.lower()
    b = b.lower()
    return list(set(a.split()) & set(b.split()))

pic = []
for patient_diag in list(third_parse.diagnosis):
    person_map = np.zeros(map_dim)
    for key in map_list:
        if not not check_share_lexi(key,patient_diag):
            person_map = np.logical_or(person_map,brain_label_map[key])
    pic.append(person_map)


In [137]:
len(pic)

44256

In [162]:
brain_map = third_parse.loc[:,['person_id','diagnosis_id']]
brain_map.count()

person_id       44256
diagnosis_id    44256
dtype: int64

In [163]:
img = pd.Series( (v for v in pic) )
len(img)

44256

In [165]:
brain_map = brain_map.reset_index(drop=True)

In [166]:
brain_map['map_img'] = img

In [167]:
brain_map.loc[0,:]

person_id                                                    1928
diagnosis_id                                               137519
map_img         [[[False, False, False, False, False, False, F...
Name: 0, dtype: object

In [ ]:
brain_map.to_pickle('./clint/docs/example_data/brain_map.pkz')